<a href="https://colab.research.google.com/github/AABNassim/DPPML/blob/master/Glacier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A simple hands-on exercise of glacier surface mass balance (SMB) modeling/reconstruction/forcasting using ML models in Python

## Acknowledgement

* ML in Glaciology Workshop (https://github.com/Machine-Learning-in-Glaciology-Workshop)
* Dataset from WGMS, ERA5-Land and other authors
* Jessy, Konrad, Tabea, Codrut

## This hands-on tutorial consist of four main steps

* Preprocessing
* Data exploration
* Training and testing
* SMB reconstruction or forcast

In [ ]:
!pip install -qq geopandas

In [ ]:
# Lets start with importing the required libraries
# Data wrangling
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

# Geospatial packages
import geopandas as gpd
import xarray as xr

# AI/ML packages
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.inspection import permutation_importance

# 1. Preprocessing

## 1.1. Get SMB data

* Glaciological SMB data from the World Glacier Monitoring Service (WGMS), Zurich
* Fluctuations of Glaciers (FoG) Database (https://wgms.ch/data_databaseversions/)
* Other data from: Garg et al., 2021, STOTEN (https://doi.org/10.1016/j.scitotenv.2021.149533)

In [ ]:
# Data
df = pd.read_csv('https://transfer.sh/4rd58J/WH_MB_1980_2020_GLAC_ELA.csv')
df.head()

* Temporal coverage of the SMB data is 41 years (1980-2020)

## 1.2. SMB data processing

In [ ]:
# Remove first two rows which are not (smb values) needed
df_smb = df.iloc[2:]
print('Total no. of SMB data points =', df_smb.count().sum())

* Also, we need to reshape the dataframe from wide format (shown above) to long format using `pd.melt()`
* We reshape the dataframe by `Year` column as identifier variables

In [ ]:
df_smb = pd.melt(df_smb, id_vars =['Year'], value_vars = df_smb.columns)

# Names of ‘variable’ and ‘value’ columns can be customized
df_smb = df_smb.rename(columns={'variable': 'RGIId', 'value': 'smb'})
df_smb["smb"] = pd.to_numeric(df_smb["smb"]) # convert "smb" column to numeric
df_smb

## 1.3. Get glacier topographical data

* We use the glacier `RGIId` (Randolph Glacier Inventory) or `GLIMSId` (Global Land Ice Measurements from Space initiative) to extract these data from the inventory

In [ ]:
# We transpose the column names (RGIId) as row/index to find the particular observation from RGI 6.0
df_topo = df.transpose() 
df_topo = df_topo.iloc[1:,:2]
df_topo.columns = ['Glacier_name', 'SMB_method']
df_topo

In [ ]:
# Open RGI inventory for 14. South Asia (West) shapefiles to extract info. for the selected glaciers
gdf = gpd.read_file('https://transfer.sh/cgChks/rgi6_SAW_polygons.gpkg')
gdf = gdf.to_crs(epsg=4326)
gdf.set_index("RGIId", inplace = True)
gdf['RGIId'] = gdf.index

#### RGI Regions

In [ ]:
Image("https://transfer.sh/irLRAF/RGI_Tech_Report_V6.0.pdf.png")

In [ ]:
# Get RGI columns/features of our interest
df_topo_rgi = df_topo.merge(df_topo.merge(gdf[['RGIId', 'GLIMSId','geometry','Area','CenLon','CenLat','Zmin', 'Slope']], 
                                left_index=True, right_index=True, how='left', sort=False))
df_topo_rgi

In [ ]:
# Replicating Zmin, Area, Slope columns for input (these features are constant every year)
df_topo_rgi_repli = df_topo_rgi[['RGIId', 'Zmin', 'Area', 'Slope']]
df_topo_rgi_repli = pd.DataFrame(np.repeat(df_topo_rgi_repli.values, 41, axis=0))
df_topo_rgi_repli.columns = ['RGIId', 'Zmin', 'Area', 'Slope']
df_topo_rgi_repli

* This will be required later while merging all variables/features together for training

* At this moment, we can also quickly visualise where are these glaciers geographically located in Asia

In [ ]:
# First, we need to convert the geometry (from RGI) to geodataframe using geopandas
df_topo_rgi = gpd.GeoDataFrame(df_topo_rgi, crs="EPSG:4326", geometry=df_topo_rgi.geometry)

In [ ]:
# Visualise
fig, ax = plt.subplots()
df_topo_rgi.plot(ax=ax)
plt.title('Selected glaciers in the Western Himalaya (from RGI)')
plt.show()

## 1.4. Get glacier climate data

* We use ERA5-Land reanalysis data at 9-km grid from the Climate Data Store (CDS, https://cds.climate.copernicus.eu/#!/home) 

In [ ]:
# Processed climate data file for the WH region
!wget -q https://transfer.sh/nauN37/era5l_1980_2020_all_variables_WH_TUM_winterschool2023.nc
climate_ds = xr.open_dataset('era5l_1980_2020_all_variables_WH_TUM_winterschool2023.nc')
climate_ds

#### Full name of the variables below:

* cpdd = cumulative positive degree days
* asn = forecast albedo
* es = snow evaporation
* sf = snowfall
* skt = surface skin temperature
* slhf = surface latent heat flux
* sshf = surface sensible heat flux
* smlt = snowmelt
* sp = surface pressure
* ssr = net solar radiation
* ssrd = solar radiation downward
* str = net thermal radiation
* strd = thermal radiation downward
* t2m = 2m temperature
* tp = total precipitation
* ws = wind speed

#### Seasons

* annual = 1 Oct to 30 Sept of next year
* summer = Jun-Aug (JJAS)
* Winter = Dec-Apr (DJFMA)

* A number of meteorological/climate variables/features can be generated, here are several examples,
* We generated/aggregated ~50 variables,
* The variable will act as `predictors` of the SMB for a year for a particular glacier, 
* Notice here is the `hydrological year (HY)`, to keep consistency with SMB year 
* But one might be curious how the climate varies spatially over the ROI,
* Lets quickly visualise the climate (`t2m`) of the ROI with glaciers over it

In [ ]:
# Visualise
fig, ax = plt.subplots()
climate_ds.t2m_annual[0].plot(ax=ax)
df_topo_rgi.plot(ax=ax, color='k')
plt.title('Glaciers (RGI) and Climate in 1980')
plt.show()

### Now, extract the climate data for each glacier for every HY using centroid values

In [ ]:
# We create required objects
lat = df_topo_rgi.CenLat
lon = df_topo_rgi.CenLon
name = df_topo_rgi.RGIId

# Start extracting the process for all the glacier centroids
# Code help: https://stackoverflow.com/questions/58635776/python-extract-multiple-lat-long-from-netcdf-files-using-xarray
Newdf = pd.DataFrame([])

for i,j,id in zip(lat,lon,name):
    dsloc = climate_ds.sel(latitude=i,longitude=j,method='nearest')
    DT=dsloc.to_dataframe()

    # insert the name with your preferred column title:
    DT.insert(loc=0,column="RGIId",value=id)
    Newdf=Newdf.append(DT,sort=True)

print(Newdf)

In [ ]:
# A bit more preprocessing before the data can be used for training
Newdf['HY'] = Newdf.index
Newdf = Newdf.reset_index(drop=True)
Newdf.head()

### At this moment, we have all variables (e.g., SMB, topographic data and climate data)
* Lets merge them all to make final training file

In [ ]:
# First, merge topographic features to the climate features dataframe
Newdf['Zmin'] = df_topo_rgi_repli['Zmin']
Newdf['Area'] = df_topo_rgi_repli['Area']
Newdf['Slope'] = df_topo_rgi_repli['Slope']

# Second, merge SMB values
Newdf = pd.concat([Newdf, df_smb.smb], axis=1, ignore_index=False, sort=False)

# 2. Data exploration 

* One can start with the basic statistics to see if there is any issue with the final data

In [ ]:
# Basic statistics
Newdf.describe()

In [ ]:
# Then, SMB data frquency in each HY
mb_hy = Newdf.groupby(Newdf['HY']).count()
mb_hy = mb_hy['smb']
mb_hy.plot.bar()
plt.xlabel('Year')
plt.ylabel('No. of SMB data points for training')
plt.show()

* Final processing step before feeding the data for training/testing
* Note that we have some SMB data gaps in different years, we need to remove those observations

In [ ]:
# Final data
data = Newdf[Newdf['smb'].notna()] # remove NaN rows

# In addition, we set HY and RGIId as index so that all glaciers/years are grouped 
# This will prevent any spatiotemporal infomration leakage while splitting 
data = data.set_index(['RGIId', 'HY'])
data

# 3. Training and Testing

In [ ]:
# Before feeding the data for training/testing, lets check the shape
print('The shape of our features is:', data.shape)

In [ ]:
# Make the training and target sets
X = data.drop('smb', axis=1)  # Features/training, here we selected all feature/variables except SMB which is our target
y = data['smb']  # Labels/target

In [ ]:
# Split dataset into training set and test set
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

#### Lets have a quick look at the shapes of the training/testing sets that we just built

In [ ]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

* We are ready to train a ML model
* For the sake of simplicity, we applied Random Forest Regressor model, which is an ensemble learning method for classification, regression and other tasks that operates by constructing a multitude of decision trees at training time.
* More details: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [ ]:
# Call a RFR model
rfr = RandomForestRegressor(n_estimators=100, bootstrap=True, random_state = 42) # Default parameters

# We can look at the parameters used by our current model
rfr.get_params()

* For the sake of simplicity and limited time, we do not change anything in the model parameters,
* Lets fit our training data to the model

In [ ]:
# Fit the model on training data
rfr.fit(X_train, y_train)

In [ ]:
# Lets check the model performance while training and testing (R2)
print(f"model train set performance: {rfr.score(X_train, y_train):.4f}")
print(f"model test set performance: {rfr.score(X_test, y_test):.4f}")

#### At the current configuration, the model is overfitting as both training/testing `r2` is far from each other, which should be closer.

* We can have a look at other performance scores, i.e., RMSE, MSE, etc

In [ ]:
# Check the MSE and RMSE score on training set
# predict the test data
y_pred = rfr.predict(X_train)
print('TRAINING')
print('R^2:', metrics.r2_score(y_train, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_train, y_pred))
print('Root Mean Squared Error (RMSE):', metrics.mean_squared_error(y_train, y_pred, squared=False))

# Check the MSE and RMSE score on testing set
y_pred = rfr.predict(X_test)
print('TESTING')
print('R^2:', metrics.r2_score(y_test, y_pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error (RMSE):', metrics.mean_squared_error(y_test, y_pred, squared=False))

#### We can also have a look at the original/predicted SMB data points

In [ ]:
# Predict
y_pred = rfr.predict(X_test)

# visualise the prediction using a 1:1 scatter plot
plt.figure()
plt.scatter(y_pred, y_test, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='k')
plt.xlabel('predicted SMB (m w.e.)')
plt.ylabel('original SMB (m w.e.)')

plt.annotate("r-squared = {:.2f}".format(metrics.r2_score(y_test, y_pred)), (-1.9, 0.25))
plt.annotate("mse = {:.2f}".format(metrics.mean_squared_error(y_test, y_pred))  + str(' m w.e.'), (-1.9, 0.15))
plt.annotate("rmse = {:.2f}".format(metrics.mean_squared_error(y_test, y_pred, squared=False)) + str(' m w.e.'), (-1.9, 0.05))

plt.title(label=str('RandomForestRegressor()'))
plt.grid(alpha=0.4)
plt.show()

## For the sake of simplicity and time, we did not show any `cross_validation` appraoch to improve the model or its performance, which is often used for better validation and predictions.

## Feature/predictor importances

* To know what are the meteorological/topographical drivers of SMB
* We make use of sklearn `permutation_importance()`

In [ ]:
result = permutation_importance(rfr, X_test, y_test, n_repeats=10, random_state=0, n_jobs=2)

In [ ]:
# Get the column/feature names
df_annual_train_columns = data.drop('smb', axis=1)

# Plot the importances
fig, ax = plt.subplots(figsize=(6,7))
sorted_idx = result.importances_mean.argsort()
ax.boxplot(
    result.importances[sorted_idx].T*100, vert=False, labels=df_annual_train_columns.columns[sorted_idx]
)
ax.set_title("Permutation importance of each feature/variable")
ax.set_ylabel("features")
ax.set_xlabel("importance [%]")
fig.tight_layout()
plt.show()

### Now, our model is ready to forcast or predic SMB for glaciers where we do not have any observation.

* We can get features from ERA5-Land or RGI or any other reanalysis/gridded open datasets, i.e., HAR, CMIP6, etc

# 4. Glacier SMB Reconstruction/Forcasting

* First, we use this model where we have some observe data and see how the model works there,
* Chhota Shigri Glacier - a benchmark glacier in the HKH

In [ ]:
temp_df = data.reset_index()
cs_smb  = temp_df[temp_df.RGIId == 'RGI60-14.15990'] # We select the Chhota Shigri Glacier, where observations are available
cs_smb = cs_smb.set_index(['RGIId', 'HY'])
cs_smb.head()

In [ ]:
# Predict SMB for Chhota Shigri Glacier
cs_smb_pred = cs_smb.iloc[:,:-1] # We dropped the observed SMB column to feed it into the model
smb_pred = rfr.predict(cs_smb_pred)

# Merge the SMB values with the feature/RGIId dataframe
cs_smb['smb_pred'] = smb_pred
cs_smb.head()

In [ ]:
# Plot and see
cs_prediction = cs_smb.reset_index()

plt.figure()
plt.plot(cs_prediction.HY, cs_prediction.smb, label = 'observation', marker='o')
plt.plot(cs_prediction.HY, cs_prediction.smb_pred, label = 'ML model', marker='o')
plt.axhline(0, linestyle='--', alpha=0.4)

plt.annotate("Mean obs. = {:.2f}".format(cs_prediction.smb.mean()) + str(' m w.e.'), (2003, 0.7))
plt.annotate("Mean ML mod. = {:.2f}".format(cs_prediction.smb_pred.mean()) + str(' m w.e.'), (2003, 0.6))

plt.xlabel('Year')
plt.ylabel('Annual SMB (m w.e.)')
plt.legend(loc='lower left')
plt.show()

* Second, lets select 10 random glaciers from the Western Himalaya to reconstruct their mass balance with the model that we trained.

In [ ]:
# Prediction data, preprocessed for random 10 glaciers
df_prediction = pd.read_csv('https://transfer.sh/DL2Dxn/TUMwinterschool_ML_SMB_Glacier_predictions.csv')
df_prediction = df_prediction.set_index(['RGIId', 'HY'])
df_prediction

In [ ]:
# Predict
smb_pred = rfr.predict(df_prediction)

# Merge the SMB values with the feature/RGIId dataframe
df_prediction['smb_pred'] = smb_pred
df_prediction

* We can plot them to look at their values

In [ ]:
df_prediction_rgiid = df_prediction.reset_index()

# Using seaborn we can quickly plot all the glacier SMB values together 
sns.lineplot(data=df_prediction_rgiid, x='HY', y='smb_pred', hue='RGIId', alpha=0.4, palette='viridis')
plt.show()

# Individual plots
# df_prediction.reset_index().groupby('RGIId').plot('HY', 'smb_pred')

# Thank you! 

# We are happy to have further discussion on model improvement, other ideas, etc.

# Resources
* Steiner, D., Walter, A., and Zumbühl, H.: The application of a non-linear back-propagation neural network to study the mass balance of Grosse Aletschgletscher, Switzerland, J. Glaciol., 51, 313–323, https://doi.org/10.3189/172756505781829421, 2005.

* Bolibar, J., Rabatel, A., Gouttevin, I., Galiez, C., Condom, T., and Sauquet, E.: Deep learning applied to glacier evolution modelling, The Cryosphere, 14, 565–584, https://doi.org/10.5194/tc-14-565-2020, 2020

* Anilkumar, R., Bharti, R., Chutia, D., and Aggarwal, S. P.: Modelling the Point Mass Balance for the Glaciers of Central European Alps using Machine Learning Techniques, EGUsphere [preprint], https://doi.org/10.5194/egusphere-2022-1076, 2022.